# Email Drafting Agent Description
Generates contextual sales emails (follow-up, introduction, proposal) using advanced prompt engineering with personalized content based on opportunity stage, account details, and engagement history.

- Generates 5 types of sales emails (introduction, follow-up, proposal, check-in, closing)
- Personalizes content based on account data, opportunity stage, and engagement history
- Supports 4 different tones (professional, friendly, urgent, consultative)
- Can generate emails in bulk

## Step 1: Import Packages

In [18]:
import json
import pandas as pd
import os
from typing import Dict, Optional, List
from datetime import datetime

from dotenv import load_dotenv

import google.generativeai as genai

## Step 2: Setup for Gemini and API Key

In [19]:
# Load environment variables .env
load_dotenv()

# Get the Gemini API key from .env
api_key = os.getenv('GEMINI_API_KEY')

In [20]:
# Configure Gemini with API key
genai.configure(api_key=api_key)

# Debug: Print available models
print("Available models:", [m.name for m in genai.list_models()])

# Create the model
model = genai.GenerativeModel('models/gemini-pro-latest')


Available models: ['models/embedding-gecko-001', 'models/gemini-2.5-pro-preview-03-25', 'models/gemini-2.5-flash', 'models/gemini-2.5-pro-preview-05-06', 'models/gemini-2.5-pro-preview-06-05', 'models/gemini-2.5-pro', 'models/gemini-2.0-flash-exp', 'models/gemini-2.0-flash', 'models/gemini-2.0-flash-001', 'models/gemini-2.0-flash-exp-image-generation', 'models/gemini-2.0-flash-lite-001', 'models/gemini-2.0-flash-lite', 'models/gemini-2.0-flash-lite-preview-02-05', 'models/gemini-2.0-flash-lite-preview', 'models/gemini-2.0-pro-exp', 'models/gemini-2.0-pro-exp-02-05', 'models/gemini-exp-1206', 'models/gemini-2.0-flash-thinking-exp-01-21', 'models/gemini-2.0-flash-thinking-exp', 'models/gemini-2.0-flash-thinking-exp-1219', 'models/gemini-2.5-flash-preview-tts', 'models/gemini-2.5-pro-preview-tts', 'models/learnlm-2.0-flash-experimental', 'models/gemma-3-1b-it', 'models/gemma-3-4b-it', 'models/gemma-3-12b-it', 'models/gemma-3-27b-it', 'models/gemma-3n-e4b-it', 'models/gemma-3n-e2b-it', 'mo

## Step 3: Load Data

In [21]:
# Load clean data
base_path = "data_directory/clean_data"

# Read CSV files
accounts = pd.read_csv(os.path.join(base_path, "Accounts.csv"))
pipeline = pd.read_csv(os.path.join(base_path, "Pipeline.csv"))
teams = pd.read_csv(os.path.join(base_path, "Teams.csv"))
products = pd.read_csv(os.path.join(base_path, "Products.csv"))

## Step 4: Email Drafting Agent Class
The EmailDraftingAgent class handles all email generation logic with prompt engineering.

**Methods:**
- `draft_email()` - Main method to generate a single email
- `generate_bulk_emails()` - Generate multiple emails in batch
- `_build_email_prompt()` - Constructs the AI prompt with context
- `_get_email_type_guidance()` - Provides type-specific instructions
- `_get_tone_instructions()` - Provides tone-specific instructions

### EmailDraftingAgent Class - Initialization

In [22]:
class EmailDraftingAgent:
    """
    Email Drafting Agent that generates contextual sales emails using advanced prompt engineering.
    Supports follow-up, introduction, and proposal email types with personalization.
    """
    
    def __init__(self, model_name="gemini-2.5-flash"):
        """
        Initialize the Email Drafting Agent.
        
        Args:
            model_name (str): The Gemini model to use for generation
        """
        self.model = genai.GenerativeModel(model_name)
        
        # Supported email types
        self.email_types = {
            "follow_up": "Follow-up Email",
            "introduction": "Introduction Email", 
            "proposal": "Proposal Email",
            "check_in": "Check-in Email",
            "closing": "Deal Closing Email"
        }
        
        print(f"EmailDraftingAgent initialized with model: {model_name}")

# Test initialization
test_agent = EmailDraftingAgent()
print(f"Supported email types: {list(test_agent.email_types.keys())}")

EmailDraftingAgent initialized with model: gemini-2.5-flash
Supported email types: ['follow_up', 'introduction', 'proposal', 'check_in', 'closing']


### EmailDraftingAgent Class - Main Draft Method

In [23]:
# Add the main draft_email method to the EmailDraftingAgent class

def draft_email(
    self,
    email_type: str,
    account_data: Dict,
    opportunity_data: Optional[Dict] = None,
    engagement_history: Optional[List[str]] = None,
    custom_context: Optional[str] = None,
    tone: str = "professional"
) -> str:
    """
    Generate a contextual sales email based on provided data.
    
    Args:
        email_type (str): Type of email ('follow_up', 'introduction', 'proposal', 'check_in', 'closing')
        account_data (dict): Account information (sector, revenue, employees, location, etc.)
        opportunity_data (dict): Opportunity details (product, deal_stage, close_value, dates, etc.)
        engagement_history (list): Recent engagement activities/interactions
        custom_context (str): Additional context or special instructions
        tone (str): Email tone ('professional', 'friendly', 'urgent', 'consultative')
        
    Returns:
        str: Generated email content
    """
    
    # Validate email type
    if email_type not in self.email_types:
        raise ValueError(f"Invalid email type. Choose from: {list(self.email_types.keys())}")
    
    # Build the comprehensive prompt
    prompt = self._build_email_prompt(
        email_type,
        account_data,
        opportunity_data,
        engagement_history,
        custom_context,
        tone
    )
    
    # Generate the email using Gemini
    response = self.model.generate_content(prompt)
    return response.text

# Add method to class
EmailDraftingAgent.draft_email = draft_email
print("draft_email method added")

draft_email method added


### EmailDraftingAgent Class - Prompt Builder

In [24]:
# Add the prompt building method to the EmailDraftingAgent class

def _build_email_prompt(
    self,
    email_type: str,
    account_data: Dict,
    opportunity_data: Optional[Dict],
    engagement_history: Optional[List[str]],
    custom_context: Optional[str],
    tone: str
) -> str:
    """
    Build an advanced prompt for email generation using structured prompt engineering.
    This is where the magic happens - crafting the perfect instructions for the AI.
    """
    
    # Get current date for context
    current_date = datetime.now().strftime("%B %d, %Y")
    
    # Build engagement history section
    engagement_section = ""
    if engagement_history and len(engagement_history) > 0:
        engagement_section = f"""
**Recent Engagement History:**
{self._format_engagement_history(engagement_history)}
"""
    
    # Build opportunity section
    opportunity_section = ""
    if opportunity_data:
        opportunity_section = f"""
**Opportunity Details:**
```json
{json.dumps(opportunity_data, indent=2)}
```
"""
    
    # Build custom context section
    context_section = ""
    if custom_context:
        context_section = f"""
**Additional Context:**
{custom_context}
"""
    
    # Get email-type-specific guidance
    type_guidance = self._get_email_type_guidance(email_type, opportunity_data)
    
    # Get tone-specific instructions
    tone_instructions = self._get_tone_instructions(tone)
    
    # Construct the main prompt with advanced prompt engineering techniques
    prompt = f"""
You are an expert B2B sales communication specialist with 15+ years of experience crafting high-converting sales emails.

**YOUR TASK:**
Generate a {self.email_types[email_type]} for a sales opportunity. The email must be personalized, contextually relevant, and action-oriented.

**EMAIL TYPE GUIDANCE:**
{type_guidance}

**TONE & STYLE:**
{tone_instructions}

**ACCOUNT INFORMATION:**
```json
{json.dumps(account_data, indent=2)}
```

{opportunity_section}

{engagement_section}

{context_section}

**CRITICAL REQUIREMENTS:**

1. **Personalization:**
   - Reference specific account details (industry, company size, location)
   - Mention relevant engagement history or previous interactions
   - Tailor value proposition to their sector and business needs
   - Use the recipient's company name naturally throughout

2. **Structure:**
   - Subject Line: Compelling, specific, and relevant (50-60 characters)
   - Opening: Personal and context-aware (reference specific details)
   - Body: Clear value proposition with industry-specific benefits
   - Call-to-Action: Specific, time-bound, and easy to act upon
   - Closing: Professional with clear next steps
   
3. **Content Quality:**
   - Use concrete data points when available (revenue, deal size, etc.)
   - Include social proof or relevant case studies for their industry
   - Address potential pain points specific to their sector
   - Keep paragraphs short (2-3 sentences max)
   - Avoid generic templates or clichés
   - Use active voice and action verbs

4. **Business Context:**
   - Acknowledge the current deal stage if applicable
   - Reference timeline/urgency appropriately
   - Highlight mutual benefits and ROI
   - Show understanding of their business challenges

**OUTPUT FORMAT:**

Subject: [Your compelling subject line]

---

Dear [Appropriate greeting],

[Email body with 3-4 concise paragraphs]

[Clear call-to-action]

[Professional closing]

[Signature block]

---

**IMPORTANT:** 
- Do NOT use placeholder brackets like [Company Name] - use actual data provided
- Be specific and data-driven, not generic
- Make it sound natural and human, not robotic
- Current date is {current_date} - use for time-sensitive context

Generate the email now:
"""
    
    return prompt

# Add method to class
EmailDraftingAgent._build_email_prompt = _build_email_prompt
print("_build_email_prompt method added")

_build_email_prompt method added


### EmailDraftingAgent Class - Helper Methods

In [25]:
# Add helper methods for email type guidance, tone instructions, and formatting

def _get_email_type_guidance(self, email_type: str, opportunity_data: Optional[Dict]) -> str:
    """
    Provide specific guidance based on email type and opportunity stage.
    """
    
    guidance = {
        "follow_up": f"""
This is a FOLLOW-UP email after a previous interaction.
- Reference the last touchpoint or conversation
- Provide value or new information since last contact
- Gently remind them of next steps or pending decisions
- Show momentum and progress on the deal
- Include a soft deadline or timeframe if appropriate
{f"- Current deal stage: {opportunity_data.get('deal_stage', 'Unknown')}" if opportunity_data else ""}
""",
        
        "introduction": """
This is an INTRODUCTION email to a new prospect.
- Hook them immediately with a relevant pain point or opportunity
- Briefly explain who you are and why you're reaching out
- Focus on their potential gains, not your product features
- Demonstrate that you've done research on their company
- Keep it concise - respect their time
- Low-pressure call-to-action (exploratory call or information sharing)
""",
        
        "proposal": f"""
This is a PROPOSAL email presenting a solution.
- Clearly articulate the business problem you're solving
- Present your solution with specific, quantifiable benefits
- Include pricing or investment information if available
- Reference their specific needs and how you address them
- Provide a clear path forward with timeline
- Create urgency through value, not pressure
{f"- Deal value: ${opportunity_data.get('close_value', 'TBD')}" if opportunity_data else ""}
""",
        
        "check_in": f"""
This is a CHECK-IN email to maintain relationship.
- Keep tone warm and consultative, not pushy
- Provide valuable insights or industry updates
- Ask about their current priorities or challenges
- Offer help without expecting immediate return
- Reaffirm your commitment to their success
{f"- Current stage: {opportunity_data.get('deal_stage', 'Nurturing')}" if opportunity_data else ""}
""",
        
        "closing": f"""
This is a CLOSING email to finalize the deal.
- Celebrate the progress made together
- Recap key benefits and ROI they'll receive
- Address any remaining concerns or questions
- Provide clear next steps and timeline
- Create positive momentum toward signature
- Include all necessary documentation or links
{f"- Deal value: ${opportunity_data.get('close_value', 'TBD')}" if opportunity_data else ""}
"""
    }
    
    return guidance.get(email_type, "")


def _get_tone_instructions(self, tone: str) -> str:
    """
    Provide tone-specific writing instructions.
    """
    
    tone_map = {
        "professional": """
- Maintain a polished, business-formal tone
- Use industry terminology appropriately
- Be respectful and courteous throughout
- Balance warmth with professionalism
""",
        "friendly": """
- Write in a warm, conversational manner
- Use contractions to sound more natural
- Show personality while staying professional
- Build rapport through relatable language
""",
        "urgent": """
- Create appropriate sense of urgency without being pushy
- Emphasize time-sensitive opportunities or deadlines
- Use action-oriented language
- Be direct and clear about what's needed
""",
        "consultative": """
- Position yourself as a trusted advisor
- Ask thoughtful questions about their business
- Demonstrate expertise through insights
- Focus on problem-solving, not selling
"""
    }
    
    return tone_map.get(tone, tone_map["professional"])


def _format_engagement_history(self, history: List[str]) -> str:
    """
    Format engagement history into a readable bullet list.
    """
    formatted = []
    for item in history:
        formatted.append(f"- {item}")
    return "\n".join(formatted)


# Add methods to class
EmailDraftingAgent._get_email_type_guidance = _get_email_type_guidance
EmailDraftingAgent._get_tone_instructions = _get_tone_instructions
EmailDraftingAgent._format_engagement_history = _format_engagement_history

print("Helper methods added")
print("EmailDraftingAgent class is now complete!")

Helper methods added
EmailDraftingAgent class is now complete!


### EmailDraftingAgent Class - Bulk Generation

In [26]:
# Add bulk email generation method

def generate_bulk_emails(
    self,
    email_configs: List[Dict]
) -> List[Dict[str, str]]:
    """
    Generate multiple emails in batch.
    
    Args:
        email_configs (list): List of email configuration dictionaries
        
    Returns:
        list: List of dictionaries with 'config', 'email', and 'status' keys
    """
    results = []
    
    for i, config in enumerate(email_configs, 1):
        try:
            print(f"Generating email {i}/{len(email_configs)}...", end=" ")
            
            email = self.draft_email(
                email_type=config.get('email_type', 'follow_up'),
                account_data=config.get('account_data', {}),
                opportunity_data=config.get('opportunity_data'),
                engagement_history=config.get('engagement_history'),
                custom_context=config.get('custom_context'),
                tone=config.get('tone', 'professional')
            )
            
            results.append({
                'config': config,
                'email': email,
                'status': 'success'
            })
            print("✓")
            
        except Exception as e:
            results.append({
                'config': config,
                'email': None,
                'status': 'error',
                'error': str(e)
            })
            print(f"✗ Error: {str(e)}")
    
    return results

# Add method to class
EmailDraftingAgent.generate_bulk_emails = generate_bulk_emails
print("Bulk generation method added")

Bulk generation method added


## Step 5: Testing

In [27]:
# Create an instance of the Email Drafting Agent
agent = EmailDraftingAgent(model_name="gemini-2.5-flash")

print("Agent ready for testing!")
print(f"Supported email types: {list(agent.email_types.keys())}")

EmailDraftingAgent initialized with model: gemini-2.5-flash
Agent ready for testing!
Supported email types: ['follow_up', 'introduction', 'proposal', 'check_in', 'closing']


### Test 1: Introduction Email

Generate an introduction email for a new prospect using actual account data.

In [28]:
# Select a sample account from your data
sample_account = accounts.sample(1).iloc[0]

# Prepare account data
account_data_test1 = {
    "account_name": sample_account['account'],
    "sector": sample_account['sector'],
    "revenue": sample_account['revenue'],
    "employees": sample_account['employees'],
    "office_location": sample_account['office_location']
}

print("Testing with account:")
print(f"  Name: {account_data_test1['account_name']}")
print(f"  Sector: {account_data_test1['sector']}")
print(f"  Revenue: ${account_data_test1['revenue']:,.0f}")
print(f"  Employees: {account_data_test1['employees']}")
print(f"\nGenerating introduction email...\n")

# Generate introduction email
intro_email = agent.draft_email(
    email_type="introduction",
    account_data=account_data_test1,
    tone="friendly"
)

print("="*80)
print("GENERATED INTRODUCTION EMAIL")
print("="*80)
print(intro_email)
print("="*80)

Testing with account:
  Name: singletechno
  Sector: retail
  Revenue: $2,215
  Employees: 5374.0

Generating introduction email...

GENERATED INTRODUCTION EMAIL
Subject: Thought on singletechno's Retail Efficiency & Growth

Hello,

I was recently looking at singletechno's impressive scale as a US retailer, particularly your operations supporting 5374 employees and over $2.2 billion in annual revenue. It's clear singletechno is a significant player, and as we approach end-of-year planning for 2026, I imagine optimizing customer experience and operational efficiency are top priorities.

Many large retailers, especially those with your footprint across the United States, often grapple with challenges like streamlining inventory management across diverse locations, personalizing customer journeys at scale, or maximizing ROI from existing technology investments. We've found these can significantly impact both the bottom line and customer loyalty.

My name is Alex Chen, and I help retail le

### Test 2: Follow-up Email with Opportunity

Generate a follow-up email that includes opportunity details and engagement history.

In [29]:
# Checking what deal stages there are
print(pipeline['deal_stage'].unique())
print(f"\nDeal stage counts:")
print(pipeline['deal_stage'].value_counts())
print("\n" + "-"*80 + "\n")

# Get an opportunity from pipeline
# Try to get an 'engaging' stage opportunity first
active_opps = pipeline[pipeline['deal_stage'] == 'engaging']

# If no 'engaging' opportunities, try 'prospecting' (if it exists)
if active_opps.empty:
    active_opps = pipeline[pipeline['deal_stage'] == 'prospecting']

# If still empty, just use 'won' as fallback
if active_opps.empty:
    print("Note: No 'engaging' or 'prospecting' opportunities found.")
    print("Using a 'won' opportunity for demonstration.\n")
    active_opps = pipeline[pipeline['deal_stage'] == 'won']

# If still empty, use any opportunity
if active_opps.empty:
    print("Note: Using random opportunity.\n")
    active_opps = pipeline

# Sample one opportunity
sample_opp = active_opps.sample(1).iloc[0]
sample_account2 = accounts[accounts['account'] == sample_opp['account']].iloc[0]

# Prepare data
account_data_test2 = {
    "account_name": sample_account2['account'],
    "sector": sample_account2['sector'],
    "revenue": sample_account2['revenue'],
    "employees": sample_account2['employees'],
    "office_location": sample_account2['office_location']
}

opportunity_data_test2 = {
    "opportunity_id": sample_opp['opportunity_id'],
    "product": sample_opp['product'],
    "deal_stage": sample_opp['deal_stage'],
    "close_value": sample_opp['close_value'],
    "engage_date": str(sample_opp['engage_date'])
}

engagement_history_test2 = [
    f"Initial contact on {sample_opp['engage_date']}",
    f"Product demo for {sample_opp['product']}",
    "Sent pricing proposal",
    "Awaiting feedback from decision maker"
]

print("Testing with:")
print(f"  Account: {account_data_test2['account_name']}")
print(f"  Opportunity: {opportunity_data_test2['opportunity_id']}")
print(f"  Product: {opportunity_data_test2['product']}")
print(f"  Stage: {opportunity_data_test2['deal_stage']}")
print(f"  Value: ${opportunity_data_test2['close_value']:,.0f}")
print(f"\nEngagement History:")
for item in engagement_history_test2:
    print(f"    - {item}")
print("\n" + "-"*80 + "\n")

print("Generating follow-up email...\n")

# Generate follow-up email
followup_email = agent.draft_email(
    email_type="follow_up",
    account_data=account_data_test2,
    opportunity_data=opportunity_data_test2,
    engagement_history=engagement_history_test2,
    tone="professional"
)

print("="*80)
print("GENERATED FOLLOW-UP EMAIL")
print("="*80)
print(followup_email)
print("="*80)

['won' 'engaging' 'lost' 'prospecting']

Deal stage counts:
deal_stage
won            4238
lost           2473
engaging       1589
prospecting     500
Name: count, dtype: int64

--------------------------------------------------------------------------------

Testing with:
  Account: hottechi
  Opportunity: 8y4ljoj0
  Product: gtx_plus_basic
  Stage: engaging
  Value: $472

Engagement History:
    - Initial contact on 2017-07-24
    - Product demo for gtx_plus_basic
    - Sent pricing proposal
    - Awaiting feedback from decision maker

--------------------------------------------------------------------------------

Generating follow-up email...

GENERATED FOLLOW-UP EMAIL
Subject: Revisiting gtx_plus_basic: An Update for hottechi

---

Dear hottechi Leadership,

I hope this email finds you well. I'm reaching out today, drawing on our previous discussions from late 2017, where we explored the capabilities of gtx_plus_basic for hottechi following a product demo and pricing proposal. A 

### Test 3: Proposal Email with Custom Context

Generate a proposal email with additional custom context about the deal.

In [30]:
# Get a high-value opportunity
high_value_opps = pipeline[pipeline['close_value'] > pipeline['close_value'].median()]
sample_opp3 = high_value_opps.sample(1).iloc[0]
sample_account3 = accounts[accounts['account'] == sample_opp3['account']].iloc[0]

# Prepare data
account_data_test3 = {
    "account_name": sample_account3['account'],
    "sector": sample_account3['sector'],
    "revenue": sample_account3['revenue'],
    "employees": sample_account3['employees'],
    "office_location": sample_account3['office_location']
}

opportunity_data_test3 = {
    "product": sample_opp3['product'],
    "deal_stage": "Proposal",
    "close_value": sample_opp3['close_value'],
    "engage_date": str(sample_opp3['engage_date'])
}

custom_context_test3 = """
Client is comparing 3 vendors including us.
Budget approved for Q4 implementation.
Key concerns: Integration time and training requirements.
Decision timeline: End of month.
"""

print("Testing with:")
print(f"  Account: {account_data_test3['account_name']}")
print(f"  Sector: {account_data_test3['sector']}")
print(f"  Deal Value: ${opportunity_data_test3['close_value']:,.0f}")
print(f"  Product: {opportunity_data_test3['product']}")
print(f"\nGenerating proposal email...\n")

# Generate proposal email
proposal_email = agent.draft_email(
    email_type="proposal",
    account_data=account_data_test3,
    opportunity_data=opportunity_data_test3,
    custom_context=custom_context_test3,
    tone="consultative"
)

print("="*80)
print("GENERATED PROPOSAL EMAIL")
print("="*80)
print(proposal_email)
print("="*80)

Testing with:
  Account: ron-tech
  Sector: medical
  Deal Value: $5,006
  Product: gtxpro

Generating proposal email...

GENERATED PROPOSAL EMAIL
Subject: ron-tech: gtxpro Proposal for Optimized Q4 Implementation

---

Dear ron-tech Team,

Following our recent discussions and considering ron-tech's significant presence in the medical sector with over 6,800 employees across the United States, I'm pleased to present our formal proposal for gtxpro. We understand that your key concerns revolve around seamless integration time and comprehensive training requirements, especially with a Q4 implementation budget already approved.

At its core, gtxpro is designed to tackle these exact challenges for large, complex organizations like ron-tech. Our solution minimizes integration friction, enabling rapid deployment within your existing medical infrastructure. We've built an intuitive platform that significantly reduces the learning curve, ensuring your teams are proficient quickly and can leverag

### Test 4: Bulk Email Generation

Generate multiple emails at once for different accounts.

In [ ]:
# Prepare configurations for 3 emails
print("Preparing 3 email configurations...\n")

sample_accounts_bulk = accounts.sample(3)
email_configs = []

for idx, account in sample_accounts_bulk.iterrows():
    # Get opportunity for this account if exists
    account_opps = pipeline[pipeline['account'] == account['account']]
    
    config = {
        'email_type': 'follow_up',
        'account_data': {
            'account_name': account['account'],
            'sector': account['sector'],
            'revenue': account['revenue'],
            'employees': account['employees'],
            'office_location': account['office_location']
        },
        'tone': 'professional'
    }
    
    # Add opportunity data if exists
    if not account_opps.empty:
        opp = account_opps.iloc[0]
        config['opportunity_data'] = {
            'product': opp['product'],
            'deal_stage': opp['deal_stage'],
            'close_value': opp['close_value']
        }
    
    email_configs.append(config)
    print(f"Config {len(email_configs)}: {account['account']}")

print("GENERATING BULK EMAILS")

# Generate emails in bulk
results = agent.generate_bulk_emails(email_configs)

# Display results
print("BULK GENERATION RESULTS")

for i, result in enumerate(results, 1):
    print(f"\n--- EMAIL {i}: {result['config']['account_data']['account_name']} ---")
    print(f"Status: {result['status']}")
    
    if result['status'] == 'success':
        print("\n" + result['email'][:500] + "...\n[truncated]")
    else:
        print(f"Error: {result.get('error', 'Unknown error')}")
    
    print("-" * 80)

print(f"\nBulk generation complete: {sum(1 for r in results if r['status'] == 'success')}/{len(results)} successful")

Preparing 3 email configurations...

Config 1: goodsilron
Config 2: betatech
Config 3: rundofase

GENERATING BULK EMAILS

Generating email 1/3... ✓
Generating email 2/3... ✓
Generating email 3/3... ✓

BULK GENERATION RESULTS


--- EMAIL 1: goodsilron ---
Status: success

Subject: goodsilron: Your GTX Plus Basic Activation & Next Steps

---

Dear [Recipient Name],

We are thrilled to formally welcome goodsilron to the GTX Plus Basic family following the successful close of your agreement on November 18, 2025. This marks an exciting new chapter for your marketing operations, and we're eager to see the impact GTX Plus Basic will have on your team of over 5000 professionals across the United States.

GTX Plus Basic is designed to empower leading marketing organizati...
[truncated]
--------------------------------------------------------------------------------

--- EMAIL 2: betatech ---
Status: success

Subject: Follow-up: `mg_special` and betatech's Medical Sector Needs

---

Dear [Recipi

### Test 5: Different Tones Comparison

Generate the same email with different tones to see how the agent adapts.

In [ ]:
# Use same account for all tones
sample_account_tone = accounts.sample(1).iloc[0]

account_data_tone = {
    "account_name": sample_account_tone['account'],
    "sector": sample_account_tone['sector'],
    "revenue": sample_account_tone['revenue'],
    "employees": sample_account_tone['employees'],
    "office_location": sample_account_tone['office_location']
}

tones = ['professional', 'friendly', 'urgent', 'consultative']

print(f"Testing all tones with: {account_data_tone['account_name']}")

for tone in tones:
    print(f"TONE: {tone.upper()}")
    
    email = agent.draft_email(
        email_type="introduction",
        account_data=account_data_tone,
        tone=tone
    )
    
    print(email)
    print("\n")

Testing all tones with: rangreen


TONE: PROFESSIONAL

Subject: Insight for rangreen: Scaling Global Technology Operations

---

Dear [Recipient Name],

My name is [Your Name], and I lead the [Your Role/Team] at [Your Company]. I'm reaching out after researching rangreen's impressive footprint in the technology sector, particularly your significant global team of nearly 8,800 employees and strategic operations based out of Panama.

For technology innovators like rangreen, managing efficient scaling while continuously driving innovation across diverse global regions can present unique challenges. Optimizing R&D pipelines, ensuring consistent operational delivery, and maintaining a competitive edge are constant strategic priorities.

We specialize in partnering with technology leaders to address these exact complexities. Our solutions enable companies to streamline their operational frameworks and accelerate their time-to-market for new solutions. For example, we recently helped a global

## Save Agent
Save the agent configuration for later use

In [33]:
import pickle

In [ ]:
# Save agent settings
agent_config = {
    'model_name': 'gemini-2.5-flash',
    'email_types': agent.email_types,
    'creation_date': datetime.now().isoformat()
}

# Saved to use in other notebooks
with open('email_agent_config.pkl', 'wb') as f:
    pickle.dump(agent_config, f)

print("Agent configuration saved to 'email_agent_config.pkl'")

Agent configuration saved to 'email_agent_config.pkl'

Agent is ready for integration with Gradio interface!
